<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-openai-agents/blob/main/mcp-agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP Agent
[code](https://replit.com/@matt/OpenAI-Agents-SDK#main.py)
[video](https://www.youtube.com/watch?v=Ta5J_2KFBGM)
[replit](https://docs.replit.com/getting-started/intro-replit)

In [ ]:
# Install

!pip install openai-agents-sdk

In [ ]:
# Imports

from agents.mcp import MCPServerStdio
import asyncio
from datetime import datetime
from agents import Agent, Runner, function_tool

In [ ]:
# Code

@function_tool
async def get_time() -> str:
  return datetime.now().strftime("%Y-%m-%d %H:%M:%S")


mcp_fetch = MCPServerStdio(params={
    "command": "uvx",
    "args": ["mcp-server-fetch"],
})


async def async_main():
  async with mcp_fetch:
    agent = Agent(name="Assistant",
                  model="gpt-4.1-mini",
                  instructions="You are a helpful assistant",
                  mcp_servers=[mcp_fetch],
                  tools=[get_time])

    result = await Runner.run(
        agent, """
            Please get the content of docs.replit.com/updates and summarize them.
            Return the summary as well as the time you got the content.
            """)

    print(result.final_output)


if __name__ == "__main__":
  asyncio.run(async_main())
